In [ ]:
pip install elapid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00


In [ ]:
pip install geopandas

In [ ]:
pip install rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the Necessary Library Packages

import elapid as ela # Species Distribution Modeling Tools
import os # To Load the Datasets/Files from the Specified Directory
import warnings # To Ignore all the Warning Messages
import numpy as np # To Load the Numerical Python Values (Arrays)
import pandas as pd # To Load the Dataframe of the given Dataset/File
import geopandas as gpd #for Geospatial Data, specifically Vector Data and other formats such as shapefiles, GeoJSON & Geopackage files
import rasterio as rio # Reading, Writing, and Manipulating Raster Geospatial Data
import rasterio.plot as rioplot # To Visualize Raster Data
from sklearn import metrics # For Evaluation Metrics (To assess the performance)
import matplotlib as mpl # For Data Visualization
import matplotlib.pyplot as plt # For Data Visualization
from rasterio.plot import show # To Display the raster data using the show function
import ipywidgets as widgets # To use HTML Interactive Widgets
from IPython.display import display # To Control the output display of objects
import re # For Patterns, i.e., used to match and manipulate strings

# PLot Preferences
mpl.style.use('ggplot')

# Ignore the Warning Messages
warnings.filterwarnings("ignore")

# Define the color for barplots
pair_colors = ['#FFCC02', '#00458C']

In [ ]:
bio_clim='//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/'

In [ ]:
# Creates a list of bioclimate variables by iterating through files in the "bio_clim" directory and selecting only those files with the ".tif" extension
bioclimate_variables=[os.path.join(bio_clim,f) for f in os.listdir(bio_clim) if f.endswith(".tif")]


In [ ]:
# Function to extract the Bioclimatic Variable Number from the path

def extract_number(path):
    # Extract the number using regular expression
    match = re.search(r'Bio_(\d+)', path)
    if match:
        return int(match.group(1))
    return 0

# Sort the paths based on the extracted number
sorted_paths = sorted(bioclimate_variables, key=extract_number)

# Print the sorted paths
for bioclimate_variables in sorted_paths:
    print(bioclimate_variables)

# Save the sorted paths in a list
bioclimate_variables = list(sorted_paths)

//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_1_Annual_mean_Temperature_WC_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_2_Mean_Diurnal_Range_WC_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_3_Isothermality_WC_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_4_Temperature_Seasonality_WC_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_5_Maximum_Temperature_of_Warmest_Month_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_6_Minimum_Temperature_of_Coldest_Month_WC_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_7_Temperature_Annual_Range_WC_1970_2000.tif
//content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/Bio_8_Mean_Temperature

In [ ]:
# Add the Labels for Map
# Read the file path and split them, since we need only bioclim variables names

labels = []  # Initialize labels as an empty list
for i in bioclimate_variables:
    filename = os.path.basename(i)
    filename_without_extension = os.path.splitext(filename)[0]
    labels.append(filename_without_extension)
print(labels)

['Bio_1_Annual_mean_Temperature_WC_1970_2000', 'Bio_2_Mean_Diurnal_Range_WC_1970_2000', 'Bio_3_Isothermality_WC_1970_2000', 'Bio_4_Temperature_Seasonality_WC_1970_2000', 'Bio_5_Maximum_Temperature_of_Warmest_Month_1970_2000', 'Bio_6_Minimum_Temperature_of_Coldest_Month_WC_1970_2000', 'Bio_7_Temperature_Annual_Range_WC_1970_2000', 'Bio_8_Mean_Temperature_of_Wettest_Quarter_WC_1970_2000', 'Bio_9_Mean_Temperature_of_Driest_Quarter_WC_1970_2000', 'Bio_10_Mean_Temperature_of_Warmest_Quarter_WC_1970_2000', 'Bio_11_Mean_Temperature_of_Coldest_Quarter_WC_1970_2000', 'Bio_12_Annual_Precipitation_WC_1970_2000', 'Bio_13_Precipitation_of_Wettest_Month_WC_1970_2000', 'Bio_14_Precipitation_of_Driest_Month_WC_1970_2000', 'Bio_15_Precipitation_of_Seasonality_WC_1970_2000', 'Bio_16_Precipitation_of_Wettest_Quarter_WC_1970_2000', 'Bio_17_Precipitation_of_Driest_Quarter_WC_1970_2000', 'Bio_18_Precipitation_of_Warmest_Quarter_WC_1970_2000', 'Bio_19_Precipitation_of_Coldest_Quarter_WC_1970_2000']


In [ ]:
vector='/content/drive/MyDrive/Project_2/Insect Pest/Species Data/EDD Maps site/Helicoverpa(2004-2023).gpkg'

In [ ]:
# read the presence data, draw background point samples
presence = gpd.read_file(vector)

In [ ]:
presence.head()

,objectid,reporter,RecOwner,SciName,ComName,Nativity,OccStatus,Status,ObsDate,DateEnt,...,Reference,Locality,Comments,ReviewDate,Reviewer,VerifyMthd,Verified,IDCred,ReviewComm,geometry
0,2483166,Virginia Corn Earworm Monitoring Program Virgi...,State Agency,Helicoverpa zea,"corn earworm, tomato fruitworm",Native,Undetected,Negative,06-07-12,10-26-12,...,NaN,"""Ballahack Rd. (Frank Williams Farm)""","""Nightly blacklight trap catch average for week """,NaN,NaN,NaN,Verified,NaN,NaN,POINT (-76.36332 36.57359)
1,2483167,Virginia Corn Earworm Monitoring Program Virgi...,State Agency,Helicoverpa zea,"corn earworm, tomato fruitworm",Native,Detected,Positive,06-14-12,10-26-12,...,NaN,"""Ballahack Rd. (Frank Williams Farm)""","""Nightly blacklight trap catch average for week """,NaN,NaN,NaN,Verified,NaN,NaN,POINT (-76.36332 36.57359)
2,2483168,Virginia Corn Earworm Monitoring Program Virgi...,State Agency,Helicoverpa zea,"corn earworm, tomato fruitworm",Native,Detected,Positive,06-21-12,10-26-12,...,NaN,"""Ballahack Rd. (Frank Williams Farm)""","""Nightly blacklight trap catch average for week """,NaN,NaN,NaN,Verified,NaN,NaN,POINT (-76.36332 36.57359)
3,2483169,Virginia Corn Earworm Monitoring Program Virgi...,State Agency,Helicoverpa zea,"corn earworm, tomato fruitworm",Native,Detected,Positive,06-28-12,10-26-12,...,NaN,"""Ballahack Rd. (Frank Williams Farm)""","""Nightly blacklight trap catch average for week """,NaN,NaN,NaN,Verified,NaN,NaN,POINT (-76.36332 36.57359)
4,2483170,Virginia Corn Earworm Monitoring Program Virgi...,State Agency,Helicoverpa zea,"corn earworm, tomato fruitworm",Native,Detected,Positive,07-12-12,10-26-12,...,NaN,"""Ballahack Rd. (Frank Williams Farm)""","""Nightly blacklight trap catch average for week """,NaN,NaN,NaN,Verified,NaN,NaN,POINT (-76.36332 36.57359)


# Bio_1_Annual_mean_Temperature_WC

In [ ]:
background = ela.sample_raster(bioclimate_variables[0], count=10_000)
background.head()

0     POINT (-92.02296 45.97994)
1     POINT (-98.18234 27.87166)
2     POINT (-93.68766 36.17821)
3     POINT (-87.19534 44.31863)
4    POINT (-115.66166 45.64767)
dtype: geometry

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

,geometry,class
0,POINT (-76.36332 36.57359),1
1,POINT (-76.36332 36.57359),1
2,POINT (-76.36332 36.57359),1
3,POINT (-76.36332 36.57359),1
4,POINT (-76.36332 36.57359),1


In [ ]:
merged[merged['class']==1]

,geometry,class
0,POINT (-76.36332 36.57359),1
1,POINT (-76.36332 36.57359),1
2,POINT (-76.36332 36.57359),1
3,POINT (-76.36332 36.57359),1
4,POINT (-76.36332 36.57359),1
...,...,...
87351,POINT (-77.34260 40.53120),1
87352,POINT (-77.18190 41.01370),1
87353,POINT (-77.88197 40.75967),1
87354,POINT (-90.15129 36.40342),1


In [ ]:
merged[merged['class']==0]

,geometry,class
87356,POINT (-92.02296 45.97994),0
87357,POINT (-98.18234 27.87166),0
87358,POINT (-93.68766 36.17821),0
87359,POINT (-87.19534 44.31863),0
87360,POINT (-115.66166 45.64767),0
...,...,...
97351,POINT (-109.50228 32.02494),0
97352,POINT (-95.85176 42.32505),0
97353,POINT (-99.68057 48.13964),0
97354,POINT (-103.34290 37.50726),0


In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

,geometry,class,b01,b02,b03,b04,b05,b06,b07,b08,...,b10,b11,b12,b13,b14,b15,b16,b17,b18,b19
0,POINT (-76.36332 36.57359),1,15.106531,10.870521,34.689091,766.439697,30.7075,-0.6295,31.337,24.063334,...,24.445499,5.544417,1251.0,140.0,77.0,17.533957,384.0,265.0,380.0,294.0
1,POINT (-76.36332 36.57359),1,15.106531,10.870521,34.689091,766.439697,30.7075,-0.6295,31.337,24.063334,...,24.445499,5.544417,1251.0,140.0,77.0,17.533957,384.0,265.0,380.0,294.0
2,POINT (-76.36332 36.57359),1,15.106531,10.870521,34.689091,766.439697,30.7075,-0.6295,31.337,24.063334,...,24.445499,5.544417,1251.0,140.0,77.0,17.533957,384.0,265.0,380.0,294.0
3,POINT (-76.36332 36.57359),1,15.106531,10.870521,34.689091,766.439697,30.7075,-0.6295,31.337,24.063334,...,24.445499,5.544417,1251.0,140.0,77.0,17.533957,384.0,265.0,380.0,294.0
4,POINT (-76.36332 36.57359),1,15.106531,10.870521,34.689091,766.439697,30.7075,-0.6295,31.337,24.063334,...,24.445499,5.544417,1251.0,140.0,77.0,17.533957,384.0,265.0,380.0,294.0


In [ ]:
# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

KeyboardInterrupt: ignored

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc1 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc1:0.3f}")



In [ ]:
# write the model predictions to disk
output_raster = 'Annual_mean_Temperature_WC_Prediction1.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)

In [ ]:
# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) &\n Bio_1_Annual_mean_Temperature_WC')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_2_Mean_Diurnal_Range_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[1], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc2 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc2:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Mean_Diurnal_Range_WC_Prediction2.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:
# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_2_Mean_Diurnal_Range_WC')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_3_Isothermality_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[2], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc3 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc3:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Isothermality_WC_Prediction3.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)

In [ ]:
# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_3_Isothermality_WC')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_4_Temperature_Seasonality_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[3], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc4 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc4:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Temperature_Seasonality_WC4.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_4_Temperature_Seasonality_WC')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_5_Maximum_Temperature_of_Warmest_Month_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[4], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc5 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc5:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Maximum_Temperature_of_Warmest_Month_WC_Prediction5.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)

In [ ]:
   # plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_5_Maximum_Temperature_of_Warmest_Month')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_6_Minimum_Temperature_of_Coldest_Month_WC.tif


In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[5], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc6 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc6:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Minimum_Temperature_of_Coldest_Month_WC_Prediction6.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:
 # plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_6_Minimum_Temperature_of_Coldest_Month')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_7_Temperature_Annual_Range_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[6], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc7 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc7:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Temperature_Annual_Range_WC_Prediction7.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_7_Temperature_Annual_Range')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_8_Mean_Temperature_of_Wettest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[7], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc8 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc8:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Mean_Temperature_of_Wettest_Quarter_WC_Prediction8.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

In [ ]:
# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_8_Mean_Temperature_of_Wettest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_9_Mean_Temperature_of_Driest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[8], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc9 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc9:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Mean_Temperature_of_Driest_Quarter_WC_Prediction9.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_9_Mean_Temperature_of_Driest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_10_Mean_Temperature_of_Warmest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[9], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc10 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc10:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Mean_Temperature_of_Warmest_Quarter_WC_Prediction10.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_10_Mean_Temperature_of_Warmest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_11_Mean_Temperature_of_Coldest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[10], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc11 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc11:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Mean_Temperature_of_Coldest_Quarter_WC_Prediction11.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)

In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_11_Mean_Temperature_of_Coldest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_12_Annual_Precipitation_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[11], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc12 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc12:0.3f}")

In [ ]:

# write the model predictions to disk
output_raster = 'Annual_Precipitation_WC_Prediction12.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_12_Annual_Precipitation_WC')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_13_Precipitation_of_Wettest_Month_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[12], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc13 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc13:0.3f}")

In [ ]:

# write the model predictions to disk
output_raster = 'Precipitation_of_Wettest_Month_WC_Prediction13.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_13_Precipitation_of_Wettest_Month')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_14_Precipitation_of_Driest_Month_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[13], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc14 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc14:0.3f}")

In [ ]:

# write the model predictions to disk
output_raster = 'Precipitation_of_Driest_Month_WC_Prediction14.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_14_Precipitation_of_Driest_Month')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_15_Precipitation_of_Seasonality_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[14], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc15 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc15:0.3f}")

In [ ]:

# write the model predictions to disk
output_raster = 'Precipitation_of_Seasonality_WC_Prediction15.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_15_Precipitation_of_Seasonality')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_16_Precipitation_of_Wettest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[15], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc16 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc16:0.3f}")

In [ ]:


# write the model predictions to disk
output_raster = 'Precipitation_of_Wettest_Quarter_WC_Prediction16.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) and \nPrecipitation_of_Wettest_Quarter_WC')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_17_Precipitation_of_Driest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[16], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc17 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc17:0.3f}")

In [ ]:

# write the model predictions to disk
output_raster = 'Precipitation_of_Driest_Quarter_WC_Prediction17.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_16_Precipitation_of_Wettest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_18_Precipitation_of_Warmest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[17], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc18 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc18:0.3f}")

In [ ]:

# write the model predictions to disk
output_raster = 'Precipitation_of_Warmest_Quarter_WC_Prediction18.tif'
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_18_Precipitation_of_Warmest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

# Bio_19_Precipitation_of_Coldest_Quarter_WC.tif

In [ ]:
# Create a Random Geographic Sample Points based on the tiff file/ Select from unmasked loaction if there is no data and Count - generate the random sample
background = ela.sample_raster(bioclimate_variables[18], count=10_000)
background.head()

In [ ]:
# merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background, add_class_label=True)
merged.head()

In [ ]:
annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
annotated.head()

# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [ ]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

In [ ]:
# evaluate training performance
ypred = model.predict(x)
auc19 = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc19:0.3f}")

In [ ]:
# write the model predictions to disk
output_raster = 'Precipitation_of_Coldest_Quarter_WC_Prediction19.tif'

#Applies a trained model to a list of raster datasets
ela.apply_model_to_rasters(model, bioclimate_variables, output_raster, quiet=True)

# and read into memory
with rio.open(output_raster, 'r') as src:
    pred = src.read(1, masked=True)


In [ ]:

# plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Helicoverpa(2004-2023) \nBio_19_Precipitation_of_Coldest_Quarter')
ax.set_xticks([])
ax.set_yticks([])
cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
plt.tight_layout()

In [ ]:
# Clears the output in IPython display
from IPython.display import clear_output


def plot_surface(bio_clim):
    global presence  # Add this line to tell Python that 'presence' is a global variable

    fig, ax = plt.subplots(figsize=(6, 6))

    with rio.open(bio_clim, 'r') as src:
        profile = src.profile.copy()

        # Low temperatures are dark grey, high temperatures are light grey
        rioplot.show(src, ax=ax, cmap="turbo")

        # Filter the species data to include only points within the extent of the bioclimatic background
        presence = presence.cx[src.bounds[0]:src.bounds[2], src.bounds[1]:src.bounds[3]]

        # Overlay the stingbug data
        presence.to_crs(src.crs).plot(column='scientific', ax=ax, legend=True)

    plt.show()


# Define the "on_file_selection" Function to select the BioClim Variable
def on_file_selection(change):
    if change.new: # checks if a new file has been selected
        bio_clim = change.new
        with output_area:  # It allows controlling where the output is displayed
            clear_output(wait=True)
            # Extract the module name from the file path
            module_name = os.path.basename(bio_clim)
            print(f"Title: {module_name}")  # Print the selected module name
            plot_surface(bio_clim) # Call the plot_surface function

# Create the dropdown widget with raster names
file_widget = widgets.Dropdown(options=bioclimate_variables, description='Select a file')

# Display the file selection widget and output area
output_area = widgets.Output()

# Display the file selection widget and output area
output_area = widgets.Output()

# Call the plot_surface function when file(s) are selected
file_widget.observe(on_file_selection, names='value')

display(file_widget, output_area)

In [ ]:
#List of output raster filenames
output_rasters = [
    'Annual_mean_Temperature_WC_Prediction1.tif',
    'Mean_Diurnal_Range_WC_Prediction2.tif',
    'Isothermality_WC_Prediction3.tif',
    'Temperature_Seasonality_WC4.tif',
    'Maximum_Temperature_of_Warmest_Month_WC_Prediction5.tif',
    'Minimum_Temperature_of_Coldest_Month_WC_Prediction6.tif',
    'Temperature_Annual_Range_WC_Prediction7.tif',
    'Mean_Temperature_of_Wettest_Quarter_WC_Prediction8.tif',
    'Mean_Temperature_of_Driest_Quarter_WC_Prediction9.tif',
    'Mean_Temperature_of_Warmest_Quarter_WC_Prediction10.tif',
    'Mean_Temperature_of_Coldest_Quarter_WC_Prediction11.tif',
    'Annual_Precipitation_WC_Prediction12.tif',
    'Precipitation_of_Wettest_Month_WC_Prediction13.tif',
    'Precipitation_of_Driest_Month_WC_Prediction14.tif',
    'Precipitation_of_Seasonality_WC_Prediction15.tif',
    'Precipitation_of_Wettest_Quarter_WC_Prediction16.tif',
    'Precipitation_of_Driest_Quarter_WC_Prediction17.tif',
    'Precipitation_of_Warmest_Quarter_WC_Prediction18.tif',
    'Precipitation_of_Coldest_Quarter_WC_Prediction19.tif'
] + [f'{i+1}_WC_Prediction.tif' for i in range(19)]



In [ ]:
# Create a grid layout for subplots
fig, axes = plt.subplots(10, 2, figsize=(15, 30), dpi=100)

# Iterate over each subplot and plot the suitability predictions
for i, ax in enumerate(axes.flat):
    # Check if the output raster file exists
    if i < len(output_rasters) and os.path.exists(output_rasters[i]):
        # Read the suitability predictions for the corresponding bioclimate variable
        with rio.open(output_rasters[i], 'r') as src:
            pred = src.read(1, masked=True)

        # Plot the suitability predictions
        plot = ax.imshow(pred, vmin=0, vmax=1, cmap='YlGnBu_r')

        # Set the custom title for the subplot
        ax.set_title(labels[i])

        ax.set_xticks([])
        ax.set_yticks([])
        cbar = plt.colorbar(plot, ax=ax, label="relative occurrence probability", pad=0.03)
    else:
        # If the output raster is missing, display a placeholder message
        ax.axis('off')
        ax.text(0.5, 0.5, 'Missing Data', horizontalalignment='center', verticalalignment='center',
                transform=ax.transAxes, fontsize=12, color='red')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# List of known labels or variables
known_labels = [
    'Annual_mean_Temperature_WC_Prediction1.tif',
    'Mean_Diurnal_Range_WC_Prediction2.tif',
    'Isothermality_WC_Prediction3.tif',
    'Temperature_Seasonality_WC4.tif',
    'Maximum_Temperature_of_Warmest_Month_WC_Prediction5.tif',
    'Minimum_Temperature_of_Coldest_Month_WC_Prediction6.tif',
    'Temperature_Annual_Range_WC_Prediction7.tif',
    'Mean_Temperature_of_Wettest_Quarter_WC_Prediction8.tif',
    'Mean_Temperature_of_Driest_Quarter_WC_Prediction9.tif',
    'Mean_Temperature_of_Warmest_Quarter_WC_Prediction10.tif',
    'Mean_Temperature_of_Coldest_Quarter_WC_Prediction11.tif',
    'Annual_Precipitation_WC_Prediction12.tif',
    'Precipitation_of_Wettest_Month_WC_Prediction13.tif',
    'Precipitation_of_Driest_Month_WC_Prediction14.tif',
    'Precipitation_of_Seasonality_WC_Prediction15.tif',
    'Precipitation_of_Wettest_Quarter_WC_Prediction16.tif',
    'Precipitation_of_Driest_Quarter_WC_Prediction17.tif',
    'Precipitation_of_Warmest_Quarter_WC_Prediction18.tif',
    'Precipitation_of_Coldest_Quarter_WC_Prediction19.tif'
]

# Initialize a dictionary to store the matching scores
matching_scores = {}

# Iterate over the known labels
for known_label in known_labels:
    # Initialize the matching score for the current known label
    matching_score = 0

    # Compare the current known label against the output raster filenames
    for output_raster in output_rasters:
        if known_label in output_raster:
            matching_score = 1
            break

    # Store the matching score in the dictionary
    matching_scores[known_label] = matching_score

# Print the matching scores
for known_label, score in matching_scores.items():
    print(f'{known_label}: {score}')


In [ ]:
auc_values = []  # Initialize an empty list to store the values

for i in range(1, 20):
    auc_values.append(f'auc{i}')  # Add each value to the list
print(auc_values)

In [ ]:
# Printing the values of the variables with formatting
for var_name in auc_values:
    value = locals().get(var_name)
    if value is not None:
        print(f"{var_name}: {value:.3f}")